<a href="https://colab.research.google.com/github/saicharan-r/EIP4/blob/master/Copy_of_WRN_A5_wide_rkg_wrn_gml_rkg_rjy_gml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')

!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
replace resized/9733.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense, Conv2D, BatchNormalization, Activation



In [0]:
import tensorflow as tf

from tqdm import tqdm_notebook as tqdm

import tensorflow.contrib.eager as tfe
#tf.enable_eager_execution()

In [13]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [14]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [15]:
one_hot_df.head()


,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
0,resized/1.jpg,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0
1,resized/2.jpg,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0
2,resized/3.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0
3,resized/4.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
4,resized/5.jpg,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0


In [0]:
import keras
import numpy as np
from google.colab.patches import cv2_imshow
from tensorflow.python.keras.utils.data_utils import Sequence

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

#class PersonDataGenerator(keras.utils.Sequence):
class PersonDataGenerator(Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True,normalize=False,aug_flow=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.normalize = normalize
        self.on_epoch_end()
        self.aug_flow=aug_flow
        #print("Shuffle = ",self.shuffle)

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        #print(batch_slice)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        #print(items["image_path"])
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        if(self.aug_flow is not None):
            image = self.aug_flow.flow(image,shuffle=False,batch_size=self.batch_size).next()
        if(self.normalize == True):
            train_mean = np.mean(image, axis=(0,1,2))
            train_std = np.std(image, axis=(0,1,2))
            #print(train_mean, train_std)
            normalize = lambda x: ((x - train_mean) / train_std).astype('float32')
            image = normalize(image)


        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [17]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [18]:
train_old_df =pd.read_csv('/content/gdrive/My Drive/WRN_Extend/train_df_wrn2_widrn_acc_1577202722.csv') 
train_df = train_old_df
val_old_df = pd.read_csv('/content/gdrive/My Drive/WRN_Extend/val_df_wrn2_widrn_acc_1577202722.csv')
val_df=val_old_df
print(train_df.shape, val_df.shape)

FileNotFoundError: ignored

In [0]:
train_df

In [0]:
from datetime import datetime
def get_curr_time():
    return int(datetime.utcnow().strftime("%s"))

model_name_itr = 'wrn2_rkg_fresh_wrn_wide_rjy2_gml2_'+str(get_curr_time())
gdrive_home_path="/content/gdrive/My Drive/WRN_Extend/"
train_csv=gdrive_home_path+"train_df_"+model_name_itr+".csv"
val_csv=gdrive_home_path+"val_df_"+model_name_itr+".csv"
json_file=gdrive_home_path+"json_"+model_name_itr+".json"
png_file=gdrive_home_path+"png_"+model_name_itr+".png"
weights_file=gdrive_home_path+"h5_"+model_name_itr+".h5"

print("Model-name:",model_name_itr)
print(train_csv,val_csv,json_file,png_file,weights_file)
train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)

# /content/gdrive/My Drive/train_df_wrn2_widrn_acc_1577202722.csv 
# /content/gdrive/My Drive/val_df_wrn2_widrn_acc_1577202722.csv 
# /content/gdrive/My Drive/json_wrn2_widrn_acc_1577202722.json
#  /content/gdrive/My Drive/png_wrn2_widrn_acc_1577202722.png 
#  /content/gdrive/My Drive/h5_wrn2_widrn_acc_1577202722.h5

In [0]:
print("Model-name:",model_name_itr)
for var_name in [train_csv,val_csv,json_file,png_file,weights_file]:
  print(var_name)

In [0]:
output_weights = ["gender_output", "imagequality_ouput","age_output", "weight_output", "bag_output", "footwear_output","emotion_output", "pose_output"]
col_splits = [_gender_cols_, _imagequality_cols_,_age_cols_, _weight_cols_, _carryingbag_cols_, _footwear_cols_, _emotion_cols_, _bodypose_cols_]
def get_dist(train_df,equalize_classes=False):
    loss_weights = {}
    index=0
    for selector_column in col_splits:
        print(selector_column)
        count = []
        percentile = []
        for age_split in selector_column:
            count.append( train_df[selector_column][train_df[age_split] == 1].shape[0])
        #print(count, np.round((count/11537.0)*100.0, 2))

        max_val = np.max(count)
        total_count = np.float32(train_df.shape[0])
        #print(count, )
        count_weights= [np.round(max_val/current_val,3) for current_val in count]
        print(count_weights)
        print(np.round((np.asarray(count)/total_count)*100.0, 2))

        #print("Top Class:",selector_column[np.argmax(count)],"Max Count",np.max(count))
        #print("Bottom Class:",selector_column[np.argmin(count)])
        #weights_dist = dict(zip(selector_column, count_weights))
        
        #print(weights_dist)
        weights_vals_dist={}
        index_val=0
        for y in range(len(count_weights)):
            weights_vals_dist[y]=count_weights[y]
            print(weights_vals_dist[y],y)
            #index_val+=index_val
            
            #loss_weights[output_weights[index]]={x,y}
        loss_weights[output_weights[index]]=weights_vals_dist
        #if equalize_classes == True:
        #    expanded_df = equalize_classwise_dist(train_df, selector_column, count)
        #    train_df = train_df.append(expanded_df, ignore_index=True)
        index+=1
    #print(loss_weights)
    return train_df,loss_weights

In [0]:
_,loss_weights_train=get_dist(train_df, equalize_classes=False)
loss_weights_train

In [0]:
train_df.head()

In [0]:
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(iter(batches))
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

In [0]:
import numpy as np
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()
        if p_1 > p:
            return input_img
        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)
            if left + w <= img_w and top + h <= img_h:
                break
        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)
        input_img[top:top + h, left:left + w, :] = c
        return input_img
    return eraser

In [0]:
# create train and validation data generators
BATCH_SIZE=16
aug_gen = ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=5,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=[0.5,2.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,2.5],
                             #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             )

train_gen = PersonDataGenerator(train_df, batch_size=BATCH_SIZE,normalize=True,aug_flow=aug_gen)
valid_gen = PersonDataGenerator(val_df, batch_size=BATCH_SIZE, shuffle=False,normalize=True)

In [0]:
CROP_LENGTH=64
train_crops = crop_generator(train_gen, CROP_LENGTH)
valid_crops = crop_generator(valid_gen, CROP_LENGTH)

In [0]:
train_gen[0]

In [0]:
def get_image_batch(data_df, batch_size=32, shuffle=True,normalize=True, selected_field='age_output'):
    new_batch = PersonDataGenerator(data_df, batch_size,shuffle, normalize)
    images, targets = next(iter(new_batch))
    num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
    labels = np.asarray([ np.argmax(targets['age_output'][pos]) for pos in range(len(targets['age_output'])) ])
    return images,labels, targets, len(images)

In [0]:
images, y_train, targets, len_train = get_image_batch(train_df, batch_size=32,normalize=True, shuffle=True)

In [0]:
images_test, y_test, targets_test, len_test = get_image_batch(val_df, batch_size=32,normalize=True, shuffle=False)

In [0]:
def print_mean_std_for_batch(datagen_process):
    image_val,target = next(iter(datagen_process))
    print(image_val.shape)
    print(np.mean(image_val.round(2), axis=(0,1,2)),np.std(image_val.round(2), axis=(0,1,2)) )
    #print(np.mean(image_val.round(2), axis=(0,1,2)),np.std(image_val.round(2), axis=(0,1,2)) )  


In [0]:
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

images_test, targets_test = next(iter(valid_gen))

print(num_units)
#print(np.mean(images.round(2), axis=(0,1,2)),np.std(images.round(2), axis=(0,1,2)) )
#print(np.mean(images_test.round(2), axis=(0,1,2)),np.std(images_test.round(2), axis=(0,1,2)) )

print_mean_std_for_batch(train_gen)
print_mean_std_for_batch(valid_gen)

In [0]:
images_test, targets_test = next(iter(valid_gen))
num_units_test = { k.split("_output")[0]:v.shape[1] for k, v in targets_test.items()}
num_units_test
#images.shape
y_test  = np.asarray([ np.argmax(targets_test['age_output'][pos]) for pos in range(len(targets_test['age_output'])) ]) ## Taking the argmax to select the correct class
cv2_imshow(cv2.resize(images_test[0], (images_test[0].shape[1], images_test[0].shape[0])))
y_test.shape
y_test[0]

In [0]:
def display_single_image(image):
    cv2_imshow(cv2.resize(image, (image.shape[1], image.shape[0])))

In [0]:
len_train, len_test = len(images), len(images_test)

# train_mean = np.mean(images, axis=(0,1,2))
# train_std = np.std(images, axis=(0,1,2))

# normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
# #pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

# images_norm = normalize(images)
# images_test_norm = normalize(images_test)


In [0]:
#cv2_imshow(cv2.resize(images_norm[0], (images_norm[0].shape[1], images_norm[0].shape[0])))
#images_norm.shape
#images_test[0].shape
display_single_image(images[10])
#display_single_image(images_test_norm[10])

In [0]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import time,math
############# Weights initializer #################
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K

In [0]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

In [0]:
#weight_decay = 0.0005
weight_decay = 0.001

def initial_conv(input):
    x = Conv2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(input)

    channel_axis = -1 #if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    return x


def expand_conv(init, base, k, strides=(1, 1)):
    x = Conv2D(base * k, (3, 3), padding='same', strides=strides, kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    channel_axis = -1 #if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = Conv2D(base * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    skip = Conv2D(base * k, (1, 1), padding='same', strides=strides, kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(init)

    m = Add()([x, skip])

    return m


def conv1_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = -1# if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Conv2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Conv2D(16 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv2_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = -1 #if K.image_dim_ordering() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Conv2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Conv2D(32 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m

def conv3_block(input, k=1, dropout=0.0):
    init = input

    channel_axis = -1 #if K.image_dim_ordering() == "th" else -1

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Activation('relu')(x)
    x = Conv2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    if dropout > 0.0: x = Dropout(dropout)(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)
    x = Conv2D(64 * k, (3, 3), padding='same', kernel_initializer='he_normal',
                      W_regularizer=l2(weight_decay),
                      use_bias=False)(x)

    m = Add()([init, x])
    return m


In [0]:
def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1,num_units_in=num_units):
    """
    Creates a Wide Residual Network with specified parameters
    :param input: Input Keras object
    :param nb_classes: Number of output classes
    :param N: Depth of the network. Compute N = (n - 4) / 6.
              Example : For a depth of 16, n = 16, N = (16 - 4) / 6 = 2
              Example2: For a depth of 28, n = 28, N = (28 - 4) / 6 = 4
              Example3: For a depth of 40, n = 40, N = (40 - 4) / 6 = 6
    :param k: Width of the network.
    :param dropout: Adds dropout if value is greater than 0.0
    :param verbose: Debug info to describe created WRN
    :return:
    """
    channel_axis = -1 #if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    nb_conv = 4

    x = expand_conv(x, 16, k)
    nb_conv += 2

    for i in range(N - 1):
        x = conv1_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 32, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv2_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = expand_conv(x, 64, k, strides=(2, 2))
    nb_conv += 2

    for i in range(N - 1):
        x = conv3_block(x, k, dropout)
        nb_conv += 2

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(x)
    x = Activation('relu')(x)

    x = AveragePooling2D((8, 8))(x)
    x = Flatten()(x)

    #x = Dense(nb_classes, W_regularizer=l2(weight_decay), activation='softmax')(x)
    output_layers = [Dense(num_units[name], activation="softmax", name=f"{name}_output")(x) for name in num_units_in.keys()]
    model = Model(ip, output_layers)

    #if verbose: print("Wide Residual Network-%d-%d created." % (nb_conv, k))
    return model
    #return x

def add_final_layer(model,num_units_in):
    output_vals = [Dense(num_units[name], activation="softmax", name=f"{name}_output")(model) for name in num_units_in.keys()]
    return output_vals

def create_model(dropout=0.05):
    from keras.utils import plot_model
    from keras.layers import Input
    from keras.models import Model

    init = (224, 224, 3)

    #wrn_28_10 = create_wide_residual_network(init, nb_classes=10, N=2, k=2, dropout=0.0)
    wrn_28_10 = create_wide_residual_network(init, nb_classes=10, N=2, k=4, dropout=dropout)
    return wrn_28_10
    #output_layers = add_final_layer(wrn_28_10_backbone, num_units)
    #model = Model(Input(shape=init), output_layers)
    #wrn_28_10.summary()

    #plot_model(wrn_28_10, "WRN-16-2.png", show_shapes=True, show_layer_names=True)

In [0]:
tf.compat.v1.disable_eager_execution()

In [0]:
wrn_28_10.losses

In [0]:
loss_weights_train

In [0]:
# wrn_28_10.compile(
#     optimizer=SGD(lr=0.5),
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     # loss_weights=loss_weights, 
#     metrics=["accuracy"]
# )
wrn_28_10=create_model()
wrn_28_10.compile(
    optimizer=SGD(lr=0.049203925),
    #,momentum=MOMENTUM, nesterov=True),
    #optimizer=SGD(lr=0.01191919191919192),
    loss=tf.keras.losses.CategoricalCrossentropy(),     
    weighted_metrics=["accuracy"]
)

In [0]:
#Min: 12.520438 0.57330394
#Min: 12.54081 0.586987
#Min: 12.628042 0.18920188
#del wrn_28_10
#Min: 7.8906326 1.3513402
def iterate_lr_finder(start_lr=0.1,end_lr=1):
  
    model=create_model()
    loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}
    model.compile(
        optimizer=SGD(lr=0.049203925),
        #,momentum=MOMENTUM, nesterov=True),
        #optimizer=SGD(lr=0.01191919191919192),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        loss_weights=loss_weights_compile,  
        #metrics=["accuracy"]
        weighted_metrics=["accuracy"]
    )


    lr_finder = LRFinder_new(model)
    lr_finder.find_generator(train_gen,start_lr=start_lr, end_lr=end_lr,epochs=10,steps_per_epoch=20)#,class_weight=loss_weights_train)
    #lr_finder.find_generator(train_gen,start_lr=0.0001, end_lr=1,epochs=10,steps_per_epoch=20)

    print("#start_lr",start_lr,"end_lr",end_lr)
    print("#Max:", np.max(lr_finder.losses),lr_finder.lrs[np.argmax(lr_finder.losses)])
    print("#Min:", np.min(lr_finder.losses), lr_finder.lrs[np.argmin(lr_finder.losses)])
    
    del model
    return lr_finder

In [0]:
lr_1 = iterate_lr_finder(start_lr=0.1,end_lr=1)

In [0]:
lr_1.plot_loss(n_skip_end=2)

In [0]:
lr_2 = iterate_lr_finder(start_lr=0.001,end_lr=0.1)

In [0]:
lr_3 = iterate_lr_finder(start_lr=0.0001,end_lr=0.001)

In [0]:
print(lr_finder.get_best_lr(1,1,1))
print(lr_finder.get_best_lr(10,1,1))
print(lr_finder.get_best_lr(20,1,1))

In [0]:
lrs_to_losses = pd.DataFrame({'lrs':lr_finder.lrs, 'losses':lr_finder.losses})
lrs_to_losses.loc[lrs_to_losses.shape[0]-1]
sma_10_lr = lr_finder.get_best_lr(10,1,1)
lrs_to_losses.loc[lrs_to_losses[lrs_to_losses['lrs'].between(sma_10_lr*0.1, sma_10_lr) ]['losses'].idxmin()]

In [0]:
lrs_to_losses

In [0]:
np.min(lr_finder.losses)

In [0]:
print("Max:", np.max(lr_finder.losses),lr_finder.lrs[np.argmax(lr_finder.losses)])
print("Min:", np.min(lr_finder.losses), lr_finder.lrs[np.argmin(lr_finder.losses)])

In [0]:
lr_finder.plot_loss(n_skip_end=1)
#lr_finder.plot_loss(n_skip_end=1,x_scale='linear')

In [0]:
for (k,v) in num_units.items():
    print(k,v)

In [0]:
#wrn_28_10=create_model()
from keras.models import load_model
# import the necessary packages
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os

# class LossHistory(keras.callbacks.Callback):
# 	def on_train_begin(self, logs={}):
# 		print("Clearing saved content on training start")
# 		self.losses = []
# 		self.best = np.Inf

class TrainingMonitor(BaseLogger):
	def __init__(self, figPath, jsonPath=None, startAt=0, backup_hist=True):
		# store the output path for the figure, the path to the JSON
		# serialized file, and the starting epoch
		super(TrainingMonitor, self).__init__()
		self.figPath = figPath
		self.jsonPath = jsonPath
		self.startAt = startAt
		self.backup_hist = backup_hist
		print("JSON path:",self.jsonPath)

	def on_train_begin(self, logs={}):
		# initialize the history dictionary
		self.H = {}
		#self.losses = []

		# if the JSON history path exists, load the training history
		if self.jsonPath is not None:
			if os.path.exists(self.jsonPath) and (self.backup_hist == True):
				#self.H = json.loads(open(self.jsonPath).read())
				backup_file_name=self.jsonPath+str(get_curr_time())+"_backup"
				print("Backing up history file:",self.jsonPath," to:",backup_file_name)
				os.rename(self.jsonPath,backup_file_name) 

				# # check to see if a starting epoch was supplied
				# if self.startAt > 0:
				# 	# loop over the entries in the history log and
				# 	# trim any entries that are past the starting
				# 	# epoch
				# 	for k in self.H.keys():
				# 		self.H[k] = self.H[k][:self.startAt]

	def on_epoch_end(self, epoch, logs={}):
		# loop over the logs and update the loss, accuracy, etc.
		# for the entire training process
		for (k, v) in logs.items():
			l = self.H.get(k, [])
			l.append(float(v))
			self.H[k] = l

		# check to see if the training history should be serialized
		# to file
		if self.jsonPath is not None:
			f = open(self.jsonPath, "w")
			f.write(json.dumps(self.H))
			f.close()
	def on_train_end(self, logs={}):
		backup_file_name=self.jsonPath+str(get_curr_time())+"_backup"
		#print("Backing up history file:",self.jsonPath," to:",backup_file_name)
		os.rename(self.jsonPath,backup_file_name) 
		print("Current JSON PATH:",self.jsonPath)
		print("Final JSON PATH:",backup_file_name)		

import os
plotPath = png_file
jsonPath = json_file
print(plotPath,jsonPath)

In [0]:
from datetime import datetime
# Prepare model model saving directory.
import os
save_dir = os.path.join('/content/gdrive/', 'My Drive')

model_name = 'assignment5_%s_model.{epoch:03d}.h5' % (model_name_itr+"rd2")
print(model_name)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_loss',
#                              verbose=1,
# #                              save_best_only=True)

# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_loss',
#                              verbose=1,
#                              save_best_only=True,mode='min')

In [0]:
EPOCHS=50
#LEARNING_RATE=1.3513402*0.1
#LEARNING_RATE=0.18883973*0.1
#LEARNING_RATE=0.5037587*0.1
LEARNING_RATE=0.020183668*0.15
STEPS_PER_EPOCH=100
test_y = np.linspace(0,EPOCHS,EPOCHS)
x=[0, (EPOCHS+1)//5, EPOCHS]
y=[LEARNING_RATE*0.01, LEARNING_RATE, LEARNING_RATE*0.0001]
interp_lr = np.interp(test_y, x, y)
def one_lr_schedule(epoch):
    # if(epoch <= 15):
    #     print("lr:",interp_lr[epoch+84],epoch)
    #     return interp_lr[epoch+84]
    print("lr:",interp_lr[epoch],epoch)
    return interp_lr[epoch]
#interp_values = np.interp(, [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])
lr_scheduler = LearningRateScheduler(one_lr_schedule)
# callbacks = [checkpoint, lr_scheduler,TrainingMonitor(figPath=plotPath,
#                                                       jsonPath=jsonPath,startAt=2)]
callbacks = [checkpoint, clr,TrainingMonitor(figPath=plotPath,
                                                      jsonPath=jsonPath,startAt=0)]

In [0]:
save_dir = os.path.join('/content/gdrive/', 'My Drive/WRN_Extend')


def generate_new_callbacks(steps_per_epoch=50,
                           epoch_count=50,
                           min_lr=0.00001, 
                           max_lr=0.1,
                           patience=25,
                           check_point=True,
                           clr_mode='triangular',
                           clr_multiplier=4):
    model_name = 'assignment5_%s_model.{epoch:03d}.h5' % (model_name_itr+"_"+str(get_curr_time()))
    print(model_name)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filepath = os.path.join(save_dir, model_name)
    local_clr = CyclicLR(base_lr=min_lr,
                            max_lr=max_lr,
                            step_size=steps_per_epoch*clr_multiplier,
                            mode=clr_mode)

    training_mon = TrainingMonitor(figPath=plotPath,
                                   jsonPath=jsonPath,
                                   startAt=0)
    ########## Introduced after 2x100 Epochs
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                    patience=patience, 
                                                    restore_best_weights=True)
    print("Returning new callback array with steps_per_epoch=",steps_per_epoch,
          "min_lr=",min_lr,
          "max_lr=",max_lr,
          "epoch_count=",epoch_count,
          "patience=",patience
          )
    callback_array = [local_clr, early_stop,training_mon]
    if(check_point == True):
        checkpoint = ModelCheckpoint(filepath=filepath,
                                monitor='val_loss',
                                verbose=1,
                                save_best_only=True,
                                mode='min')
        callback_array.append(checkpoint)
    
    return callback_array

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git
from keras_contrib.callbacks import CyclicLR


In [0]:
loss_weights_train

In [0]:
def run_model_iterations(model,
                         re_compile=True,
                         epoch_count=50, 
                         steps_per_epoch=50, 
                         min_lr=LEARNING_RATE*0.01, 
                         max_lr=LEARNING_RATE,
                         loss_weights_compile={},
                         loss_weights_train={}                         
                         ):


    if re_compile == True:
        model.compile(
            #optimizer=SGD(lr=1.3513402*0.1),
            optimizer=SGD(lr=min_lr),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            loss_weights=loss_weights_compile,
            #weighted_metrics=["accuracy"]
            metrics=["accuracy"]
        )
    model.fit_generator(
        generator=train_gen,
        validation_data=valid_gen,
        use_multiprocessing=True,
        workers=4, 
        epochs=1,
        verbose=1,
        class_weight=loss_weights_train,
        steps_per_epoch=steps_per_epoch,
        callbacks=generate_new_callbacks(steps_per_epoch=steps_per_epoch, min_lrm=in_lr, max_lr=max_lr, epoch_count=epoch_count)
    )


In [0]:
wrn_28_10=create_model()
run_model_iterations(wrn_28_10,
                    re_compile=True,
                    epoch_count=1, 
                    steps_per_epoch=1, 
                    min_lr=LEARNING_RATE*0.01, 
                    max_lr=LEARNING_RATE,
                    loss_weights_compile={},
                    loss_weights_train={})

In [0]:
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model()
wrn_28_10=create_model()
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=30
EPOCHS=100
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}

wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

In [0]:
print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

In [0]:
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model()
wrn_28_10=create_model()
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=100
EPOCHS=100
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 4, 
#                         'weight_output': 3, 
#                         'bag_output': 3, 
#                         'pose_output': 3, 
#                         'footwear_output': 2, 
#                         'emotion_output': 4}



wrn_28_10.load_weights('/content/gdrive/My Drive/assignment5_wrn2_rkg_fresh_wrn_1577517949rd2_model.097.h5')
wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

In [0]:
print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

In [0]:
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model()
STEPS_PER_EPOCH=200
EPOCHS=100

#wrn_28_10=create_model()
wrn_28_10.load_weights('/content/gdrive/My Drive/assignment5_wrn2_rkg_fresh_wrn_1577517949rd2_model.097.h5')
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}




wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

In [0]:
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model()
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE
EPOCHS=30

wrn_28_10=create_model()
wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_1577517949rd2_model.096.h5')
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}

wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

In [0]:
wrn_28_10.save("/content/gdrive/My Drive/WRN_Extend/model_8233acc.h5py")

In [0]:
aug_gen_array = [ ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=5,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=[0.5,2.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,2.5],
                             #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             ),
           ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=30,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=[0.5,2.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,2.5],
                             #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             ),
           ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=5,
                             width_shift_range=0.5,
                             height_shift_range=0.5,
                             zoom_range=[0.5,2.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,2.5],
                             #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             ),
           ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=5,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=[0.5,3.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,2.5],
                             #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             ),
           ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=5,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=[0.5,3.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,2.5],
                             #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             ),
           ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=5,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=[0.5,2.5],
                             shear_range=0.8,
                             #zca_whitening=True,
                             brightness_range=[0.5,4.5],
                             #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             ),
           ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=5,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=[0.5,2.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,2.5],
                             preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             )
]
           

In [0]:
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model()
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=30
EPOCHS=50

#wrn_28_10=create_model()
# aug_gen = ImageDataGenerator(horizontal_flip=True, 
#                              vertical_flip=False,
#                              rotation_range=5,
#                              width_shift_range=0.1,
#                              height_shift_range=0.1,
#                              zoom_range=[0.5,2.5],
#                              shear_range=0.2,
#                              #zca_whitening=True,
#                              brightness_range=[0.5,2.5],
#                              #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
#                              )
from keras.models import load_model
for aug_gen_count in range(len(aug_gen_array)):
    print("Executing augmentation",aug_gen_count)
    new_model = create_model()
    new_model.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
    
    train_gen = PersonDataGenerator(train_df, 
                                    batch_size=BATCH_SIZE,
                                    normalize=True,
                                    aug_flow=aug_gen_array[aug_gen_count])
    valid_gen = PersonDataGenerator(val_df, 
                                    batch_size=BATCH_SIZE, 
                                    shuffle=False,normalize=True)

    #wrn_28_10.load_weights()

    loss_weights_compile = {'gender_output': 2, 
                            'image_quality_output': 2, 
                            'age_output': 4, 
                            'weight_output': 3, 
                            'bag_output': 3, 
                            'pose_output': 3, 
                            'footwear_output': 2, 
                            'emotion_output': 4}

    new_model.compile(
        #optimizer=SGD(lr=1.3513402*0.1),
        optimizer=SGD(lr=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        loss_weights=loss_weights_compile,
        #weighted_metrics=["accuracy"]
        metrics=["accuracy"]
    )
    callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                        epoch_count=EPOCHS,
                                        min_lr=LEARNING_RATE*0.01, 
                                        max_lr=LEARNING_RATE,
                                        patience=10, check_point=False)
    #print(callbacks)
    new_model.fit_generator(
        generator=train_gen,
        validation_data=valid_gen,
        use_multiprocessing=True,
        workers=4, 
        epochs=EPOCHS,
        verbose=1,
        class_weight=loss_weights_train,
        steps_per_epoch=STEPS_PER_EPOCH,
        callbacks=callbacks
    )
    del new_model
    print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

    

In [0]:
###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
################# Tweaking weights #################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model()
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=30
EPOCHS=100
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
wrn_28_10=create_model()
wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 6, 
                        'weight_output': 3, 
                        'bag_output': 5, 
                        'pose_output': 4, 
                        'footwear_output': 4, 
                        'emotion_output': 4}

wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

###################### loss on weight change ################
#[-0.0207 -0.0044  0.0069  0.0108 -0.0015 -0.0098 -0.0089 -0.0103]


In [0]:
###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model()
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE
EPOCHS=100
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
wrn_28_10=create_model()
wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=15)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

In [0]:
###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model()
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=360 #train_df.shape[0]//BATCH_SIZE
EPOCHS=100
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_rjy_1577567748_1577569953_model.006.h5')
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=15)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model()
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE
EPOCHS=300
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
#wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_rjy2_1577594233_1577594387_model.024.h5')
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=50)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4,
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
import glob
def get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.json",count=2):
    #path = "/content/gdrive/My Drive/WRN_Extend/"
    file_path = os.path.join(path)+pattern
    files = [f for f in glob.iglob(file_path, recursive=False)]
    latest_file = sorted(files, key=os.path.getctime,reverse=True)
    return latest_file[:count]
#files = os.listdir(path)
#filenamere.split(r'/',files[len(files)-1])

In [0]:
get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=1)

In [0]:
########## Stopped previous run on Account of overfitting ##############
######### Start run with increased dropout

###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
wrn_28_10=create_model(dropout=0.1)
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE
EPOCHS=10
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
#wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
last_saved_file = get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=1)
wrn_28_10.load_weights(last_saved_file[0])
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=10)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4,
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
########## Stopped previous run on Account of overfitting ##############
######### Start run with increased dropout

###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model(dropout=0.1)
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE//2
EPOCHS=10
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
#wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
last_saved_file = get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=1)
print(last_saved_file[0])
wrn_28_10.load_weights(last_saved_file[0])
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=10)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4,
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
########## Stopped previous run on Account of overfitting ##############
######### Start run with increased dropout

###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model(dropout=0.1)
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE//2
EPOCHS=20
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
#wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
last_saved_file = get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=1)
print(last_saved_file[0])
wrn_28_10.load_weights(last_saved_file[0])
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=10)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4,
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=20)

In [0]:
########## Stopped previous run on Account of overfitting ##############
######### Start run with increased dropout

###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
wrn_28_10=create_model(dropout=0.05)
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE//2
EPOCHS=20
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
#wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
last_saved_file = '/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_rjy2_1577594233_1577603312_model.048.h5'
#get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=1)
print(last_saved_file)
wrn_28_10.load_weights(last_saved_file)
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=10)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4,
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
wrn_28_10.save('/content/gdrive/My Drive/WRN_Extend/Latest_val_loss_19924306.h5py')

In [0]:
########## Loss improved to 19.924306 ##############


################################# Below are multiple Trial and Errors ######################
################################# No big difference observed in the results ###############################
######### Trying with clr param changes clr_mode='triangular2', clr_multiplier=8

###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model(dropout=0.05)
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE//2
EPOCHS=20
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
#wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
last_saved_file = '/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_rjy2_gml2_1577624977_1577625577_model.008.h5'
#get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=1)
print(last_saved_file)
wrn_28_10.load_weights(last_saved_file)
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=10,
                                   clr_mode='triangular2',
                                   clr_multiplier=8)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4,
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
########## Loss improved to 19.924306 ##############
######### Triangular2 didnt give much benefit #### 
######### Fallback to triangular with multiplier 4

###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model(dropout=0.05)
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE//2
EPOCHS=30
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
#wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
last_saved_file = '/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_rjy2_gml2_1577624977_1577625577_model.008.h5'
#get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=1)
print(last_saved_file)
wrn_28_10.load_weights(last_saved_file)
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 2, 
                        'image_quality_output': 2, 
                        'age_output': 4, 
                        'weight_output': 3, 
                        'bag_output': 3, 
                        'pose_output': 3, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=10,
                                   #clr_mode='triangular2',
                                   clr_multiplier=4)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4,
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
########## Loss improved to 19.924306 ##############
######### Triangular2 didnt give much benefit #### 
######### Fallback to triangular with multiplier 4

###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model(dropout=0.05)
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE//2
EPOCHS=20
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
#wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
last_saved_file = '/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_rjy2_gml2_1577624977_1577625577_model.008.h5'
#last_saved_file ='/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_rjy2_gml2_1577624977_1577632609_model.008.h5'
#get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=1)
print(last_saved_file)
wrn_28_10.load_weights(last_saved_file)
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 1, 
                        'image_quality_output': 2, 
                        'age_output': 2, 
                        'weight_output': 1, 
                        'bag_output': 2, 
                        'pose_output': 1, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=10,
                                   #clr_mode='triangular2',
                                   clr_multiplier=4)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4,
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
########## Loss improved to 19.924306 ##############
######### Triangular2 didnt give much benefit #### 
######### Fallback to triangular with multiplier 4

###################### Couldn't find a good augmentation strategy hence sticking with defaults ##################
#LEARNING_RATE=1.3513402*0.1
#del wrn_28_10
#wrn_28_10=create_model(dropout=0.05)
LEARNING_RATE=0.2511886*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE//2
EPOCHS=20
#"/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5"
#wrn_28_10=create_model()
#wrn_28_10.load_weights('/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_1577543864_1577544293_model.022.h5')
last_saved_file = '/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_rjy2_gml2_1577624977_1577637125_model.016.h5'
#last_saved_file ='/content/gdrive/My Drive/WRN_Extend/assignment5_wrn2_rkg_fresh_wrn_wide_rjy2_gml2_1577624977_1577632609_model.008.h5'
#get_latestn_files(path = "/content/gdrive/My Drive/WRN_Extend/", pattern="*.h5",count=1)
print(last_saved_file)
wrn_28_10.load_weights(last_saved_file)
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 6, 
#                         'weight_output': 3, 
#                         'bag_output': 5, 
#                         'pose_output': 4, 
#                         'footwear_output': 4, 
#                         'emotion_output': 4}
loss_weights_compile = {'gender_output': 1, 
                        'image_quality_output': 2, 
                        'age_output': 2, 
                        'weight_output': 1, 
                        'bag_output': 2, 
                        'pose_output': 1, 
                        'footwear_output': 2, 
                        'emotion_output': 4}


wrn_28_10.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE,
                                   patience=10,
                                   #clr_mode='triangular2',
                                   clr_multiplier=4)
#print(callbacks)
wrn_28_10.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4,
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)


In [0]:
!cp "/content/gdrive/My Drive/json_wrn2_widrn_acc_1577202722.json" "/content/gdrive/My Drive/json_wrn2_widrn_acc_1577202722_round2.json"

In [0]:
wrn_28_10.save(weights_file+"py")

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred_results = wrn_28_10.predict_generator(valid_gen, (2036 // 32+1),verbose=True)

In [0]:
y_pred_dict['val_age_output_acc'].shape

In [0]:
y_pred_dict_new=get_indexed_results(y_pred_results)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
def get_indexed_results(y_pred_results):
    class_output_arr = ['gender',
                        'imagequality',
                        'age',
                        'weight',
                        'carryingbag',
                        'bodypose',    
                        'footwear',   
                        'emotion']
    y_pred_dict = {}
    for len_val in range(len(y_pred_results)):
        y_pred_local = y_pred_results[len_val][:-12]
        y_pred_final = np.argmax(y_pred_local, axis=1)
        y_pred_dict[class_output_arr[len_val]]=y_pred_final
    return y_pred_dict
def print_confusion_matrix(y_pred_dict, val_df_to_use, attribute_to_select):
    cols_to_select=[col for col in one_hot_df.columns if col.startswith(attribute_to_select)]
    y_true = val_df_to_use[cols_to_select].values
    y_true_classes = np.argmax(y_true, axis=1)
    y_pred=y_pred_dict[attribute_to_select]
    #print(y_pred_results[].shape, y_true.shape)
    #print(y_true_classes)
    y_true_classes = np.argmax(y_true, axis=1)
    print("Confusion Matrix for", attribute_to_select)
    matrix = confusion_matrix(y_true_classes, y_pred)
    print(matrix)
    true_class_dist = [ np.where( y_true_classes==classes)[0].shape[0] for classes in np.unique(y_true_classes)]
    print("True Class Dist",true_class_dist)
    pred_class_dist = [ np.where( y_pred==classes)[0].shape[0] for classes in np.unique(y_pred)]
    print("Predicted class dist",pred_class_dist)

from sklearn.metrics import classification_report, confusion_matrix
y_pred_results = wrn_28_10.predict_generator(valid_gen, (2036 // 32+1),verbose=True)

y_pred_dict_new=get_indexed_results(y_pred_results)
for value_col in range(len(class_output_arr)):
    print_confusion_matrix(y_pred_dict_new, val_df, class_output_arr[value_col])
    print("***************")

In [0]:
for value_col in range(len(class_output_arr)):
    print_confusion_matrix(y_pred_dict_new, val_df, class_output_arr[value_col])
    print("***************")

In [0]:
y_pred=y_pred_dict['val_emotion_output_acc']
true_class_dist = [ np.where( y_true_classes==classes)[0].shape[0] for classes in np.unique(y_true_classes)]
print("True Class Dist",true_class_dist)
pred_class_dist = [ np.where( y_pred==classes)[0].shape[0] for classes in np.unique(y_pred)]
print("Predicted class dist",pred_class_dist)

In [0]:
val_gender_output_acc   
val_image_quality_output_acc    
val_age_output_acc    
val_weight_output_acc    
val_bag_output_acc   
val_pose_output_acc    
val_footwear_output_acc   
val_emotion_output_acc


In [0]:
from matplotlib import pyplot as plt
import math
from keras.callbacks import LambdaCallback
import keras.backend as K
import numpy as np


class LRFinder_new:
    """
    Plots the change of the loss function of a Keras model when the learning rate is exponentially increasing.
    See for details:
    https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0
    """

    def __init__(self, model):
        self.model = model
        self.losses = []
        self.lrs = []
        self.best_loss = 1e9

    def on_batch_end(self, batch, logs):
        # Log the learning rate
        lr = K.get_value(self.model.optimizer.lr)
        self.lrs.append(lr)

        # Log the loss
        loss = logs['loss']
        self.losses.append(loss)

        # Check whether the loss got too large or NaN
        if batch > 5 and (math.isnan(loss) or loss > self.best_loss * 4):
            print("")
            print("Training stopped due to high loss",loss)
            self.model.stop_training = True
            return

        if loss < self.best_loss:
            self.best_loss = loss

        # Increase the learning rate for the next batch
        lr *= self.lr_mult
        K.set_value(self.model.optimizer.lr, lr)

    def find(self, x_train, y_train, start_lr, end_lr, batch_size=64, epochs=1):
        # If x_train contains data for multiple inputs, use length of the first input.
        # Assumption: the first element in the list is single input; NOT a list of inputs.
        N = x_train[0].shape[0] if isinstance(x_train, list) else x_train.shape[0]

        # Compute number of batches and LR multiplier
        num_batches = epochs * N / batch_size
        self.lr_mult = (float(end_lr) / float(start_lr)) ** (float(1) / float(num_batches))
        # Save weights into a file
        self.model.save_weights('tmp.h5')

        # Remember the original learning rate
        original_lr = K.get_value(self.model.optimizer.lr)

        # Set the initial learning rate
        K.set_value(self.model.optimizer.lr, start_lr)

        callback = LambdaCallback(on_batch_end=lambda batch, logs: self.on_batch_end(batch, logs))

        self.model.fit(x_train, y_train,
                       batch_size=batch_size, epochs=epochs,
                       callbacks=[callback])

        # Restore the weights to the state before model fitting
        self.model.load_weights('tmp.h5')

        # Restore the original learning rate
        K.set_value(self.model.optimizer.lr, original_lr)

    def find_generator(self, generator, start_lr, end_lr, epochs=1, steps_per_epoch=None, **kw_fit):
        if steps_per_epoch is None:
            try:
                steps_per_epoch = len(generator)
            except (ValueError, NotImplementedError) as e:
                raise e('`steps_per_epoch=None` is only valid for a'
                        ' generator based on the '
                        '`keras.utils.Sequence`'
                        ' class. Please specify `steps_per_epoch` '
                        'or use the `keras.utils.Sequence` class.')
        self.lr_mult = (float(end_lr) / float(start_lr)) ** (float(1) / float(epochs * steps_per_epoch))

        # Save weights into a file
        self.model.save_weights('tmp.h5')

        # Remember the original learning rate
        original_lr = K.get_value(self.model.optimizer.lr)

        # Set the initial learning rate
        K.set_value(self.model.optimizer.lr, start_lr)

        callback = LambdaCallback(on_batch_end=lambda batch,
                                                      logs: self.on_batch_end(batch, logs))

        self.model.fit_generator(generator=generator,
                                 epochs=epochs,
                                 steps_per_epoch=steps_per_epoch,
                                 callbacks=[callback],
                                 **kw_fit)

        # Restore the weights to the state before model fitting
        self.model.load_weights('tmp.h5')

        # Restore the original learning rate
        K.set_value(self.model.optimizer.lr, original_lr)

    def plot_loss(self, n_skip_beginning=10, n_skip_end=5, x_scale='log'):
        """
        Plots the loss.
        Parameters:
            n_skip_beginning - number of batches to skip on the left.
            n_skip_end - number of batches to skip on the right.
        """
        plt.ylabel("loss")
        plt.xlabel("learning rate (log scale)")
        plt.plot(self.lrs[n_skip_beginning:-n_skip_end], self.losses[n_skip_beginning:-n_skip_end])
        plt.xscale(x_scale)
        plt.show()

    def plot_loss_change(self, sma=1, n_skip_beginning=10, n_skip_end=5, y_lim=(-0.01, 0.01)):
        """
        Plots rate of change of the loss function.
        Parameters:
            sma - number of batches for simple moving average to smooth out the curve.
            n_skip_beginning - number of batches to skip on the left.
            n_skip_end - number of batches to skip on the right.
            y_lim - limits for the y axis.
        """
        derivatives = self.get_derivatives(sma)[n_skip_beginning:-n_skip_end]
        lrs = self.lrs[n_skip_beginning:-n_skip_end]
        plt.ylabel("rate of loss change")
        plt.xlabel("learning rate (log scale)")
        plt.plot(lrs, derivatives)
        plt.xscale('log')
        plt.ylim(y_lim)
        plt.show()

    def get_derivatives(self, sma):
        assert sma >= 1
        derivatives = [0] * sma
        for i in range(sma, len(self.lrs)):
            derivatives.append((self.losses[i] - self.losses[i - sma]) / sma)
        return derivatives

    def get_best_lr(self, sma, n_skip_beginning=10, n_skip_end=5):
        derivatives = self.get_derivatives(sma)
        best_der_idx = np.argmax(derivatives[n_skip_beginning:-n_skip_end])
        return self.lrs[n_skip_beginning:-n_skip_end][best_der_idx]
